In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import glob

from spectral_analysis_functions import *
# Using OceanDataLab reader 
#from sentinel2_l1c import *
import numpy as np
from S2_read import *
from s2_angs import *
S2file='/home/ardhuin/ADMIN/PROPOSALS/EE11/S2/DATA/T11SMS_20160429T183252';
S2path='/home/ardhuin/Téléchargements/S2A_MSIL1C_20160429T183252_N0201_R027_T11SMS_20160429T184037.SAFE'


In [ ]:
boxi=[5600,6400,3600,4400] # East West South North 
bands=['B04','B03','B02'];  
nb=np.shape(bands)[0]
[imgs,NX,NY,nx,ny,dx,dy]=S2_read(S2path,boxi,bands)


X = np.arange(0,nx*dx,dx) # from 0 to (nx-1)*dx with a dx step
Y = np.arange(0,ny*dy,dy)


In [ ]:
# normalizes the image values
img1=imgs[0,:,:]/np.mean(imgs[0,:,:].flatten())
img2=imgs[2,:,:]/np.mean(imgs[2,:,:].flatten())
plt.imshow(np.transpose(np.fliplr(img1)),vmin=0.5, vmax=2) #edit your vmin, vmax and cmap if you don't like greyscale colormap
plt.colorbar()
plt.show()
plt.imshow(np.transpose(np.fliplr(img2)),vmin=0.5, vmax=2) #edit your vmin, vmax and cmap if you don't like greyscale colormap
plt.colorbar()
plt.show()

In [ ]:
#img2=img1[0:400,0:800]
#(Eta,Eta_all,kx2,ky2,dkx,dky)=FFT2D_one_array(img1,10,10,16,isplot=0)
(Eta,Etb,ang,angstd,coh,crosr,phases,kx2,ky2,dkx,dky)=FFT2D_two_arrays(img1,img2,10,10,16,isplot=0)
print(np.shape(Eta))
print(np.shape(kx2))
print(np.shape(ky2))

fig,axs=plt.subplots(1,2,figsize=(20,7))#,sharey=True,sharex=True)
_=plt.gca().set_aspect('equal', adjustable='box')

im=axs[0].pcolormesh(X/1000,Y/1000,np.transpose(img1),cmap='seismic',norm = mcolors.Normalize(vmin=0.5, vmax=2))
_=plt.colorbar(im,ax=axs[0],label='S2 image band B04')
_=axs[0].set_xlabel('X [km]')
_=axs[0].set_ylabel('Y [km]')
_=axs[0].set_title('Surface')

im=axs[1].pcolormesh(kx2*1000,ky2*1000,10*np.log10((Eta)),norm = mcolors.Normalize(vmin=-30, vmax=20))
_=plt.colorbar(im,ax=axs[1],label='dB')

_=axs[1].set_xlabel('k_x [cycles / km]')
_=axs[1].set_ylabel('k_y [cycles / km]')
_=axs[1].set_title('Spectrum')
_=plt.tight_layout()

print('variance:',np.var(img1.flatten()), 'sum of spectrum:',np.sum(Eta.flatten())*dkx*dky )


fig,axs=plt.subplots(1,2,figsize=(20,7))#,sharey=True,sharex=True)
_=plt.gca().set_aspect('equal', adjustable='box')

im=axs[0].pcolormesh(kx2*1000,ky2*1000,coh,norm = mcolors.Normalize(vmin=0, vmax=1))
_=plt.colorbar(im,ax=axs[0],label=' ')
_=axs[1].set_xlabel('k_x [cycles / km]')
_=axs[1].set_ylabel('k_y [cycles / km]')
_=axs[0].set_title('coherence B02-B04')

im=axs[1].pcolormesh(kx2*1000,ky2*1000,np.degrees(ang),norm = mcolors.Normalize(vmin=-180, vmax=180))
_=plt.colorbar(im,ax=axs[1],label='degrees')

_=axs[1].set_xlabel('k_x [cycles / km]')
_=axs[1].set_ylabel('k_y [cycles / km]')
_=axs[1].set_title('cross-spectrum phase')
_=plt.tight_layout()

In [ ]:
nx

In [ ]:
dirC1=110
dirC2=120
dt=1
dir2a=np.degrees(np.arctan2(kx2,ky2))
kn=np.sqrt(kx2**2+ky2**2)*(2*np.pi)+1E-20
kncpk=kn/(2*np.pi)*1000


J =np.where((coh > 0.01) & (dir2a < dirC2) & (dir2a > dirC1) & (np.degrees(angstd)  < 60) )
J2=np.where((coh > 0.01) & (dir2a < dirC2) & (dir2a > dirC1) )
dispt=np.sqrt(9.81/kn)
Cimg=ang/kn/dt


plt.plot(kn[J]*1000,dispt[J],'kx',linewidth=3,label='Airy theory') 
plt.plot(kn[J]*1000,Cimg[J],'bo',linewidth=1,label='from image X-spectra') 
plt.grid(True)
plt.xlabel('k (cycles / km)')
plt.ylabel('phase speed (m/s)')
plt.legend()